In [1]:
import numpy as np #Library for advance Numpy functionality
import pandas as pd #Library to make use of Pandas Dataframes
import requests #Library to handle URL requests to get data from Web services.
import urllib.request #Libary to handle request status and data
import time #Libary to manipulate script execution time or to keep track of ellapsed time.
import json #Libary to work smartly with JSON data retrieved from API calls.
from bs4 import BeautifulSoup #Web library to make returned text legible.

In [2]:
#Initial api call to business review page on Hello Peter
url = 'https://api.hellopeter.com/consumer/business/open-serve/reviews'
response = requests.get(url)
init_dictionary = response.json()

In [3]:
#Accessing Python Dictionary to determin last page that reviews get Pagenated too.
#This is important for the upcoming loop that will do the X amount of calls to get all pages data.
pagination_lastpage =  int(init_dictionary['last_page'])+1
pagination_firstpage = 0

In [4]:
#Timing of Script
start = time.time()

#Blank list to which to write the Json in dictionary format for each page.
hp_dictionary_allpages = []

#Loop through each of the pages on Hello Peter which contains the complaints.

for x in range(pagination_firstpage,pagination_lastpage):
    
    
    #next_page_url = 'https://api.hellopeter.com/consumer/business/open-serve/reviews?page=2'"
    #So here I'm just replacing the number two with 1 through to the last page of the 
    #Pagination.
    url2 = init_dictionary['next_page_url'][:-1]+str((x+1))
    response2 = requests.get(url2)
    
    #Parsing Data as Json Dictionary and appending it tou our hp_dictionary_allpages list
    hp_dictionary_allpages.append(response2.json())
    
end = time.time()
print(str(round(end - start,0))+' Seconds Elapsed')


11.0 Seconds Elapsed


In [5]:
print('OVERALL PAGE DATA STRUCTURE:\n -----------------------------')
for key in hp_dictionary_allpages[0]:
    print(key)
    


OVERALL PAGE DATA STRUCTURE:
 -----------------------------
current_page
data
first_page_url
from
last_page
last_page_url
next_page_url
path
per_page
prev_page_url
to
total


In [6]:
print('POST DATA: \n-----------------------------')

for key in hp_dictionary_allpages[0]['data'][0]:
    print(key)

POST DATA: 
-----------------------------
id
created_at
authorDisplayName
author
email
authorAvatar
author_id
author_ip_address
review_title
review_rating
review_content
business_name
business_slug
permalink
replied
messages
business_logo
industry_logo
industry_name
industry_slug
status_id
nps_rating
is_reported


In [7]:
print(str(hp_dictionary_allpages[20]['total'])+' Hallo Peter Reviews Detected')

233 Hallo Peter Reviews Detected


In [8]:
#Creating a dataframe with all complaints info
complaints_df = pd.DataFrame.from_dict(hp_dictionary_allpages[0]['data'])
df_counter = 1
while df_counter < pagination_lastpage-1:
    temp_df = pd.DataFrame.from_dict(hp_dictionary_allpages[df_counter]['data'])
    complaints_df = complaints_df.append(temp_df,ignore_index=True)
    df_counter = df_counter + 1 

In [11]:
#df.pivot_table(index=['created_at'], aggfunc='size')
complaints_df.head()

,author,authorAvatar,authorDisplayName,author_id,author_ip_address,business_logo,business_name,business_slug,created_at,email,...,industry_slug,is_reported,messages,nps_rating,permalink,replied,review_content,review_rating,review_title,status_id
0,Chanty S,,Chanty S,5e43e7c0-81fb-11e8-8c06-f1517651de4b,105.216.59.98,None,Openserve,open-serve,2019-09-10 16:14:44,chanty@stream-line.co.za,...,internet-service-provider,False,[],NaN,openserve-has-no-client-service-technicianssup...,0,Absolutely worst service i have ever received!...,1,OPENSERVE has NO CLIENT SERVICE & Technicians/...,1
1,Karin W,,Karin W,dca56330-d180-11e9-b20b-9910337f7eae,41.13.116.156,None,Openserve,open-serve,2019-09-07 17:09:18,karin@prokuris.com,...,internet-service-provider,False,[],NaN,py-service-and-workmanship-2851185,0,"Unürpfessional and dissorganised, tool 3 weeks...",1,****py service and workmanship,1
2,Sinenhlanhla M,,Sinenhlanhla M,7cc13500-d79f-11e8-8e96-dd69d43add92,102.250.155.61,None,Openserve,open-serve,2019-09-06 12:27:48,sinenmnisi@yahoo.com,...,internet-service-provider,False,[],0,openserve-rather-not-2850487,0,They work with telkom so... yeah don’t expect ...,1,Openserve? Rather not...,1
3,Theuns T,,Theuns T,354fa630-c4e4-11e9-b041-57ba13cff6f3,102.250.4.245,None,Openserve,open-serve,2019-08-22 16:05:30,theuns.tdt@gmail.com,...,internet-service-provider,False,[],NaN,worst-technicians-ever-2837664,0,Openserve technician driving up and down in ou...,1,Worst technicians ever.,1
4,Morgan R,,Morgan R,2058521e-31fa-11e8-83f4-f23c91bb6188,41.114.166.96,None,Openserve,open-serve,2019-08-19 23:12:10,mrgnraad@gmail.com,...,internet-service-provider,False,[],7,openserve-disrespecting-property-owner-rights-...,0,Extremely unhappy on how I am being treated by...,1,Openserve disrespecting property owner rights,1
